In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
import pandas as pd
import matplotlib.animation as animation
from matplotlib.colors import LightSource
from matplotlib import cm

import statsmodels.api as sm            # to build a LOWESS model
from scipy.interpolate import interp1d  # for interpolation of new data points
lowess = sm.nonparametric.lowess

In [ ]:
## Function to compute average BMB, average BMB in deep part, and flux
def find_BMB_average(ds_hf, add_to_xarray=False, save_to_netcdf=False, savepath=None):

    # Create empty array, same length as time array
    BMB__av      = np.zeros([len(ds_hf.time.values)])
    BMB__av_deep = np.zeros([len(ds_hf.time.values)])

    # For the integrated melt flux: take BMB sum
    BMB__flux    = ds_hf.BMB.sum(dim=['x','y'], skipna=True)*2000*2000*910*10**(-12)

    # Iterate over all time steps in output
    for i,t in enumerate(ds_hf.time.values):
        BMB__av[i]      = np.nanmean(xr.where(ds_hf.sel(time=t).mask==4, ds_hf.sel(time=t)['BMB'], np.nan).values)
        BMB__av_deep[i] = np.nanmean(xr.where(np.logical_and(ds_hf.sel(time=t).mask==4, ds_hf.sel(time=t).Hs-ds_hf.sel(time=t).Hi<-300), ds_hf.sel(time=t)['BMB'], np.nan).values)

    # Add to the file if wanted
    if add_to_xarray == True:
        ds_hf = ds_hf.assign(BMB_av     =(('time'), BMB__av))
        ds_hf = ds_hf.assign(BMB_av_deep=(('time'), BMB__av_deep))
        ds_hf = ds_hf.assign(BMB_flux   =(('time'), BMB__flux.values))

        ds_BMB = ds_hf[['BMB_av', 'BMB_av_deep', 'BMB_flux']]

        # Save to netcdf file if wanted
        if save_to_netcdf == True:
            print('saving to...', savepath)
            ds_BMB.to_netcdf(savepath)

    return BMB__av, BMB__av_deep, BMB__flux

In [ ]:
# Post processing loop for MAIN runs and RETUNED runs
dir = '/Volumes/T7/P1_DIR/Model_output/NOcalving/' #'/Users/5941962/surfdrive/P1_DIR/Model_output'
exp_names = [ 

    #MAIN RUNS
    'P1_MEDIUM_QUAD_NOcalving',
    'P1_MEDIUM_PICO_NOcalving_C023', 
    'P1_MEDIUM_PLUME_NOcalving',
    'P1_MEDIUM_laddie_NOcalving_rhoice_concat',

    'P1_WARM_QUAD_NOcalving_modtune',
    'P1_WARM_PICO_NOcalving_C023_modtune', 
    'P1_WARM_PLUME_NOcalving_modtune',
    'P1_WARM_laddie_NOcalving_rhoice_concat', 

    #RETUNED RUNS
    'P1_MEDIUM_QUAD_NOcalving_hightune', 
    'P1_MEDIUM_PICO_NOcalving_C023_hightune', 
    'P1_MEDIUM_PLUME_NOcalving_hightune', 

    'P1_WARM_QUAD_NOcalving',
    'P1_WARM_PICO_NOcalving_C023', 
    'P1_WARM_PLUME_NOcalving',
]

filename = 'help_fields_ANT.nc'
combined_list = [f'{dir}/{exp_name}/{filename}' for exp_name in exp_names]


for i in range(len(combined_list)):

    print(f'Started computation BMB averages and integrated flux for {exp_names[i]} ...')

    # Open file
    ds = xr.open_dataset(f'{dir}/{exp_names[i]}/{filename}')

    # Compute exact grounding line position from f_grnd
    find_BMB_average(ds, add_to_xarray=True, save_to_netcdf=True, savepath=f'../PostProcess/BMB_{exp_names[i]}.nc')

    print(f'... DONE')
    ds.close()

In [ ]:
# Post processing loop for CONTROL & CALVING runs
dir = '/Volumes/T7/P1_DIR/Model_output/' #'/Users/5941962/surfdrive/P1_DIR/Model_output'
exp_names = [ 
    #CONTROL RUN
    'P1_control',
    
    #CALVING
    'P1_WARM_QUAD_modtune',
    'P1_WARM_PICO_C023_modtune', 
    'P1_WARM_PLUME_modtune',
    'P1_WARM_laddie_rhoice_concat', 

    'P1_MEDIUM_QUAD',
    'P1_MEDIUM_PICO_C023', 
    'P1_MEDIUM_PLUME',
    'P1_MEDIUM_laddie_rhoice_concat',
]

filename = 'help_fields_ANT.nc'
combined_list = [f'{dir}/{exp_name}/{filename}' for exp_name in exp_names]


for i in range(len(combined_list)):

    print(f'Started computation BMB averages and integrated flux for {exp_names[i]} ...')

    # Open file
    ds = xr.open_dataset(f'{dir}/{exp_names[i]}/{filename}')

    # Compute exact grounding line position from f_grnd
    find_BMB_average(ds, add_to_xarray=True, save_to_netcdf=True, savepath=f'../PostProcess/BMB_{exp_names[i]}.nc')

    print(f'... DONE')
    ds.close()